# 1898 Restaurant Data
Goal: Create a set of uniform code for cleaning up the street names and street numbers. 


In [1]:
import pandas as pd

## Master

In [2]:
DF = pd.read_csv('datasets/1880_streets_full.csv')
DF.head(1)

,street_name,odd_on,avg_direction,building_num_range,start_end_coordinates,segment_length,segment_direction,building_num_range_length,avg_length_per_building,direction_deviation,road_type,offset_from_road_center
0,10th Avenue,left,-170.561731,"[1, 19]","[[40.741073, -74.009227], [40.739952, -74.0094...",126.194091,-170.561731,19,6.641794,0.0,Avenue,10


In [3]:
DF['street_name'].value_counts()

10th Avenue         164
8th Avenue          148
2nd Avenue          129
5th Avenue          129
3rd Avenue          126
                   ... 
Manhattan Street      1
Blackwells  S         1
Bedloes Island        1
Trimble Place         1
W 178th Street        1
Name: street_name, Length: 666, dtype: int64

In [4]:
master_list = list(dict(DF['street_name'].value_counts()).keys())
#sorted(master_list)

## 1898

In [5]:
#df = pd.read_csv('datasets/1898TrowBus.csv')
df = pd.read_csv('datasets/1898Final.csv', error_bad_lines=False, encoding='latin-1')
df.head(1)

,FULL,Year,PG,ID,ID2,CAT_ORG,CAT_MOD,FULL.1,Position,Nation,...,if error,if>40,final_api,final_ancestry,country_match,region_match,Final Result,Unnamed: 51,Unnamed: 52,Unnamed: 53
0,"Aueringer Erhard, 151 E 4th",1898_TrowBusMan,951.0,127.0,1898_TrowBusMan_127,Restaurants (Marked thus + are oyster saloons.),Restaurant,"Aueringer Erhard, 151 E 4th",18.0,NaN,...,NaN,0.508446986,Austrian,German,NaN,NaN,American,NaN,NaN,NaN


In [6]:
df.shape

(2001, 54)

In [7]:
df.columns

Index(['FULL', 'Year', 'PG', 'ID', 'ID2', 'CAT_ORG', 'CAT_MOD', 'FULL.1',
       'Position', 'Nation', 'Name', 'Addy_Clean', 'Address',
       'Nationality - Katherine Vote', 'Nationality - Zehra Vote',
       'Nationality - Jessica', 'Nationality Final', 'api_result1',
       'api_result2', 'nationality_api', 'nationality_ipums', 'Origin_api',
       'Gender - Katherine Vote', 'Gender - Zehra Vote', 'Gender - Jessica',
       'Gender - Final', 'Gender_api', 'gender_combined', 'gender_mary',
       'Notes', 'Arrival / I Record', 'CORNER', 'Address2', 'ST combi',
       'ST Name', 'ST Name 2', 'ST Name 3', 'ST Name 4', 'ST Name 5',
       'ST Name 6', 'Last', 'First', 'probability1', 'probability12',
       'if error', 'if>40', 'final_api', 'final_ancestry', 'country_match',
       'region_match', 'Final Result', 'Unnamed: 51', 'Unnamed: 52',
       'Unnamed: 53'],
      dtype='object')

Get the columns I need.

In [8]:
df = df[['FULL', 'Name', 'Addy_Clean', 'ST combi', 'gender_mary', 'Final Result']]
df.head()

,FULL,Name,Addy_Clean,ST combi,gender_mary,Final Result
0,"Aueringer Erhard, 151 E 4th",Aueringer Erhard,151 E 4th,E 4th,m,American
1,"Brandstaedter Herman, 175 E Houston",Brandstaedter Herman,175 E Houston,E Houston,m,American
2,"Deixler Max, 96 Allen",Deixler Max,96 Allen st,Allen st,m,American
3,"Gerstenberger Richard G. 406, 7th av",Gerstenberger Richard G,406 7th av,7th av,m,American
4,"Greitner Herman, 1321. 3d av",Greitner Herman,1321 3rd av,3rd av,m,American


In [9]:
df.columns = ['full', 'name', 'address', 'street', 'gender', 'region']
df.head(1)

,full,name,address,street,gender,region
0,"Aueringer Erhard, 151 E 4th",Aueringer Erhard,151 E 4th,E 4th,m,American


In [10]:
#df['address'].str.split().str[0]

In [11]:
df['no'] = df['address'].str.split().str[0]
df.head()

,full,name,address,street,gender,region,no
0,"Aueringer Erhard, 151 E 4th",Aueringer Erhard,151 E 4th,E 4th,m,American,151
1,"Brandstaedter Herman, 175 E Houston",Brandstaedter Herman,175 E Houston,E Houston,m,American,175
2,"Deixler Max, 96 Allen",Deixler Max,96 Allen st,Allen st,m,American,96
3,"Gerstenberger Richard G. 406, 7th av",Gerstenberger Richard G,406 7th av,7th av,m,American,406
4,"Greitner Herman, 1321. 3d av",Greitner Herman,1321 3rd av,3rd av,m,American,1321


### Street

### systematic cleaning

#### strip unnecessary white spaces

In [12]:
df['street_clean'] = df['street'].str.strip()
df.head(1)

,full,name,address,street,gender,region,no,street_clean
0,"Aueringer Erhard, 151 E 4th",Aueringer Erhard,151 E 4th,E 4th,m,American,151,E 4th


Inspect `street_clean` data.

In [13]:
st = dict(df['street_clean'].value_counts())
#sorted(st.items())

#### add "street" after "E xxth" or "W xxth"

In [14]:
df['street_clean'].dropna(inplace=True)

In [15]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x+' Street' if x.startswith('E ') | x.startswith('W ') else x)
df.head(1)

,full,name,address,street,gender,region,no,street_clean
0,"Aueringer Erhard, 151 E 4th",Aueringer Erhard,151 E 4th,E 4th,m,American,151,E 4th Street


In [16]:
st = dict(df['street_clean'].value_counts())
#sorted(st.items())

#### replace abbreviation with full names.

In [17]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x.replace(' st', ' Street') if x.endswith(' st') else x)

In [18]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x.replace(' av', ' Avenue') if x.endswith(' av') else x)

In [19]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x.replace(' pl', ' Place') if x.endswith(' pl') else x)

In [20]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x.replace(' mkt', ' Market') if x.endswith(' mkt') else x)

In [21]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x.replace(' ln', ' Lane') if x.endswith(' ln') else x)

In [22]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x.replace(' sq', ' Square') if x.endswith(' sq') else x)

In [23]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x.replace(' slip', ' Slip') if x.endswith(' slip') else x)

In [24]:
st = dict(df['street_clean'].value_counts())
#sorted(st.items())

#### add "street" to all the ones with only one word

In [25]:
df['street_clean'] = df['street_clean'].dropna().apply(
    lambda x: x+' Street' if len(x.split())==1 else x)

In [26]:
st = dict(df['street_clean'].value_counts())
#sorted(st.items())

#### change to title case

### cross check results

In [27]:
streets_list = list(dict(df['street_clean'].value_counts()).keys())
streets_list[:5]

['3rd Avenue', 'Broadway Street', '2nd Avenue', '1st Avenue', '6th Avenue']

In [28]:
for street in streets_list:
    if street not in master_list:
        print(street)

Broadway Street
E Houston Street
W Broadway Street
Park row

Amsterdam Avenue
Columbus Avenue
Av A
W Houston Street
Fulton Market
E Broadway Street
Av C
Av B
Chatham Square
5th Street
2nd Street
1st Street
Trinity Place
Tompkins Market
N Moore
St Mark??Û?s Place
Clinton Market
8th Street
Macdougal Street
Tremont Avenue
7th Street
av c W
6th Street
Washington Market
Lincoln Avenue
3 Avenue
S William
Union sq E
Willis Avenue
Grand st Circle
N William
c 3rd Avenue
W End av Street
9th Street
Hanover Square
VandeWater Street
Centre Market
Bible h
Stone William
Catherine Market
Brook Avenue
av n W
Jerome Avenue
Jefferson Market
8d Avenue
Av D
New Chambers
E 145th Street
av n E
Little W 12th


### manual cleaning

In [29]:
STREET = {
    # street added wrongly
    '1st Street': '1st Avenue',
    '2nd Street': '2nd Avenue',
    '3 Avenue': '3rd Avenue',
    'c 3rd Avenue': '3rd Avenue',
    '5th Street': '5th Avenue',
    '6th Street': '6th Avenue',
    '7th Street': '7th Avenue',
    '8th Street': '8th Avenue',
    '8d Avenue': '8th Avenue',
    '9th Street': '9th Avenue',
    'Broadway Street': 'Broadway',
    
    # direction order
    'W Broadway Street': 'Broadway W',
    'E Broadway Street': 'Broadway E',
    'W Houston Street': 'Houston Street W',
    'E Houston Street': 'Houston Street E',
    'N Moore': 'Moore Street N',
    'S William': 'William Street S',
    'N William': 'William Street N',
    'Union sq E': 'Union Square E',
    'Little W 12th': 'W 12th Street',
    'E 145th Street': 'E 145th Street',
    
    # title case
    'Park row': 'Park Row',
    'James slip': 'James Slip',
    'Old slip': 'Old Slip',
    'Catherine slip': 'Catherine Slip',
    'Peck slip'
    
    # order
    'Av A': 'A Avenue',
    'Av B': 'B Avenue',
    'Av C': 'C Avenue',
    'av c W': 'C Avenue',
    'Av D': 'D Avenue',
    'av n E': 'East Avenue',
    'av n W': 'West Avenue',
    'W End av Street': 'West Avenue',
    
    # same name
    'Centre Market': 'Centre Market Place',
    'Clinton Market': 'Clinton Place',
    'Chatham Square': 'Chatham Street',
    'Catherine Market': 'Catherine Street',
    'Fulton Market': 'Fulton Street',
    'Grand st Circle': 'Grand Street',
    'Jefferson Market': 'Jefferson Street',
    'Stone William': 'Stone Street',
    'Tompkins Market': 'Tompkins Street',
    'Washington Market': 'Washington Place',
    'Hanover Square': 'Hanover Street',
    'New Chambers': 'New Chambers Street',
    
    # different spelling
    'Macdougal Street': 'Mac Dougal Street',
    'St Mark’s Place': 'Saint Marks Place',
    'VandeWater Street': 'Vandewater Street',
    
    # unmatched
    # Amsterdam Avenue
    # Columbus Avenue
    # Trinity Place
    # Lincoln Avenue
    # Tremont Avenue
    # Brook Avenue
    # Willis Avenue
    # Bible h
    # Jerome Avenue
}

In [30]:
df = df.replace({'street_clean': STREET})
df.head(1)

,full,name,address,street,gender,region,no,street_clean
0,"Aueringer Erhard, 151 E 4th",Aueringer Erhard,151 E 4th,E 4th,m,American,151,E 4th Street


In [31]:
streets_list = list(dict(df['street_clean'].value_counts()).keys())
streets_list[:5]

['3rd Avenue', '2nd Avenue', 'Broadway', '1st Avenue', '6th Avenue']

### cross check results

In [32]:
for street in streets_list:
    if street not in master_list:
        print(street)

Broadway W

Amsterdam Avenue
Columbus Avenue
Av A
Broadway E
St Mark??Û?s Place
Trinity Place
Tremont Avenue
Willis Avenue
Lincoln Avenue
West Avenue
East Avenue
Bible h
E 145th Street
Jerome Avenue
Brook Avenue


### House Number

In [33]:
df['num_clean'] = df['no']
df.head(1)

,full,name,address,street,gender,region,no,street_clean,num_clean
0,"Aueringer Erhard, 151 E 4th",Aueringer Erhard,151 E 4th,E 4th,m,American,151,E 4th Street,151


In [34]:
no = dict(df['num_clean'].value_counts())
#sorted(no.items())

In [35]:
df['num_clean'] = df['num_clean'].dropna().apply(
    lambda x: x.replace('th', '') if str(x).endswith('th') else x)

In [36]:
df['num_clean'] = df['num_clean'].dropna().apply(
    lambda x: x.replace('d', '') if str(x).endswith('d') else x)

In [37]:
df['num_clean'] = df['num_clean'].dropna().astype(str).astype(float).astype(int)

In [38]:
df.to_csv('cleaned/1898_address_CLEANED.csv')